In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [84]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import stdev
from statistics import mean
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
# Defining Virus tramission temperarture range(in Fahrenheit)
a=60
b=90

In [15]:
# For feature transformation of average summer temperature
avg= 23.9   # In degree celcius, and obtained from Fig8 in the "paper". 
sigma= 1.5  #23.9+2*sigma=27. It will not change with change in temp from Celcius to Kelvin.

In [16]:
# Converting Fahrenheit to Kelvin
a=(a-32)*5/9 + 273
b=(b-32)*5/9 + 273
# Converting Celciusto Kelvin
avg+=273

In [5]:
# Variable for importimng all the NARR data
# variables= ["air.2m", "air.sfc", "apcp", "rhum.2m"]
variables= ["air.2m", "apcp", "rhum.2m"]

In [6]:
months=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
years=range(1999, 2016)

In [7]:
# Importing WNV data
df= pd.read_csv("/Users/sparshagarwal/Downloads/WMV_data/Arbovirus_risk_modeling_US/WNV_human_cases/WNV_NI_NNI_1999to2015_prevalence_incidence_final_20180530.csv", encoding='latin-1')
#Selecting Urban Counties
df=df[df["Select_County"]==1]
#Removing data for California due to data anomaly
df=df[df["STNAME"]!="California"]

df.rename(columns = {'GEOID10':'GEOID'}, inplace = True)

In [8]:
# Adding a column for incident year
incident_year=[years[-1] for i in range(len(df))]
for j in range(len(df)):
    for i in years:
        if(df.iloc[j]["NI_IR_" + str(i)]!=0):
            incident_year[j]=i
            break
df["Incident_year"]=incident_year

In [9]:
# Adding temp, precp, and humidity data
df_mod=df.copy()
for i in variables:
    for j in years:
        for k in range(len(months)):
            #Importing monthly Variable data
            data= pd.read_csv("/Users/sparshagarwal/Downloads/WMV_data/intersections/narr_urban_county_data_masked/" + i +"_masked/"+ str(j) + "_" + months[k] + "_masked.csv")
            data=data[["GEOID", "mean"]]
            data.rename(columns = {'mean': i + '_' + str(j) + "_" + months[k]}, inplace = True)
            df_mod=df_mod.join(data.set_index('GEOID'), on='GEOID')

In [10]:
df_mod=df_mod[:10]   # Just for simplicity

In [30]:
# For transforming temperature into normally distributed variable
def temp_transform(temp):
    ans= (1/(np.sqrt(2*np.pi)*sigma))*(np.exp(-((temp-avg)**2)/(2* (sigma)**2)))
    return ans

In [11]:
cdd=[]        #cumulative degree days  
cp=[]         #cumulative precipitation
temp_avg=[]   #Average temperature within replication range (mean summer temp)
prec_avg=[]   #Average precipitation within replication range
hum_avg=[]    #Average humidity within replication range
temp_sd=[]    #Temperature Std.dev. within replication range
prec_sd=[]    #Precipitation Std.dev. within replication range
hum_sd=[]     #Humidity Std.dev. within replication range
temp_y_sd=[]  #Temperature yearly Std.dev. within replication range
prec_y_sd=[]  #Precipitation yearly Std.dev. within replication range
hum_y_sd=[]   #Humidity yearly Std.dev. within replication range
temp_o_avg=[] #Average temperature outside replication range
prec_o_avg=[] #Average precipitation outside replication range
hum_o_avg=[]  #Average humidity outside replication range
ir=[]         #For storing IR of WNV
geoid=[]      #For storing GEOIDs


for i in range(len(df_mod)):

    inc_year= df_mod.iloc[i]["Incident_year"]  # 1st year of WNV incidence
    
    for j in range(inc_year+1, years[-1]+1):    # For all years after first incident year
        #Number of days in february
        feb=28
        #For leap year
        if(j%4==0):
            feb=29
        #Months with number of days
        days={"01":31, "02":feb, "03" :31, "04":30, "05":31, "06":30, "07":31, "08":31, "09":30, "10":31, "11":30, "12":31}        
        
        v_cdd=0
        v_cp=0
        v_temp_avg=[]
        v_prec_avg=[]
        v_hum_avg=[]
        v_temp_y_sd=[]
        v_prec_y_sd=[]
        v_hum_y_sd=[]
        v_temp_o_avg=[]
        v_prec_o_avg=[]
        v_hum_o_avg=[]
        
        for k in months:
            v_temp_y_sd.append(df_mod.iloc[i][variables[0] + "_" + str(j) + "_" + k])
            v_prec_y_sd.append(df_mod.iloc[i][variables[1] + "_" + str(j) + "_" + k])
            v_hum_y_sd.append(df_mod.iloc[i][variables[2] + "_" + str(j) + "_" + k])
            
            if((df_mod.iloc[i][variables[0] + "_" + str(j) + "_" + k]>=a) and (df_mod.iloc[i][variables[0] + "_" + str(j) + "_" + k]<=b)):
            #if(df_mod.iloc[i][variables[0] + "_" + str(j) + "_" + k]>=a):
                v_cdd=v_cdd + days[k]
                v_cp=v_cp + df_mod.iloc[i][variables[1] + "_" + str(j) + "_" + k]
                
                v_temp_avg.append(df_mod.iloc[i][variables[0] + "_" + str(j) + "_" + k])
                v_prec_avg.append(df_mod.iloc[i][variables[1] + "_" + str(j) + "_" + k])
                v_hum_avg.append(df_mod.iloc[i][variables[2] + "_" + str(j) + "_" + k])
                
            else:
                v_temp_o_avg.append(df_mod.iloc[i][variables[0] + "_" + str(j) + "_" + k])
                v_prec_o_avg.append(df_mod.iloc[i][variables[1] + "_" + str(j) + "_" + k])
                v_hum_o_avg.append(df_mod.iloc[i][variables[2] + "_" + str(j) + "_" + k])
                
        
        # Calculating and appending std. dev., mean and values of the variables.

        temp_y_sd.append(np.std(np.array(v_temp_y_sd)))
        prec_y_sd. append(np.std(np.array(v_prec_y_sd)))
        hum_y_sd. append(np.std(np.array(v_hum_y_sd)))
      
        geoid.append(df_mod.iloc[i]["GEOID"])
        ir.append(df_mod.iloc[i]["NI_IR_" + str(j)])
        
        cdd.append(v_cdd)
        cp.append(v_cp)
        
        temp_avg.append(mean(np.array(v_temp_avg)))
        prec_avg.append(mean(np.array(v_prec_avg)))
        hum_avg.append(mean(np.array(v_hum_avg)))
        
        temp_sd.append(stdev(np.array(v_temp_avg)))
        prec_sd.append(stdev(np.array(v_prec_avg)))
        hum_sd.append(stdev(np.array(v_hum_avg)))
        
        temp_o_avg.append(mean(np.array(v_temp_o_avg)))
        prec_o_avg.append(mean(np.array(v_prec_o_avg)))
        hum_o_avg.append(mean(np.array(v_hum_o_avg)))

In [12]:
final=pd.DataFrame()
final["GEOID"]=geoid
final["IR"]=ir
final["CDD"]=cdd
final["CP"]=cp
final["Temp_avg"]=temp_avg
final["Prec_avg"]=prec_avg
final["Hum_avg"]=hum_avg
final["Temp_sd"]=temp_sd
final["Prec_sd"]=prec_sd
final["Hum_sd"]=hum_sd
final["Temp_y_sd"]=temp_y_sd
final["Prec_y_sd"]=prec_y_sd
final["Hum_y_sd"]=hum_y_sd
final["Temp_o_avg"]=temp_o_avg
final["Prec_o_avg"]=prec_o_avg
final["Hum_o_avg"]=hum_o_avg

In [ ]:
#Looking at correlation between variables (especially between IR and rest of the variables)
final.corr()

In [13]:
data=shuffle(final)

GEOID        IR  CDD         CP    Temp_avg  Prec_avg    Hum_avg  \
58   1051  0.000000  214  22.738397  296.570295  3.248342  74.320560   
16   1003  0.000000  245  26.989222  296.802467  3.373653  71.458904   
17   1003  0.000000  214  28.791416  296.992597  4.113059  76.307679   
96   1073  0.760296  245  27.882088  295.506373  3.485261  72.216269   
48   1045  0.000000  214  26.235750  297.342887  3.747964  74.432963   
..    ...       ...  ...        ...         ...       ...        ...   
9    1001  0.000000  245  27.663820  296.186375  3.457977  72.228345   
32   1015  0.000000  214  19.008305  296.153931  2.715472  69.739151   
39   1045  0.000000  214  27.305021  296.944118  3.900717  74.415750   
26   1015  0.000000  184  17.099725  296.649648  2.849954  74.204149   
15   1003  0.594810  214  19.791385  298.077737  2.827341  71.664455   

     Temp_sd   Prec_sd    Hum_sd  Temp_y_sd  Prec_y_sd  Hum_y_sd  Temp_o_avg  \
58  3.977348  2.093997  2.871647   8.014565   1.681764  3.448314  282.011603   
16  4.954854  1.579714  2.692745   6.634763   1.343343  3.969985  285.460716   
17  3.811456  1.843005  1.322877   6.562665   1.440895  2.059249  285.124390   
96  4.405391  1.638167  1.100596   6.741919   1.544500  2.801784  283.137121   
48  3.734961  2.551962  2.770085   7.562897   1.929531  3.194216  283.532025   
..       ...       ...       ...        ...        ...       ...         ...   
9   4.217097  1.729158  1.357888   6.472315   1.650138  2.634239  284.298653   
32  5.163590  1.565459  3.275725   7.977673   1.556199  3.909864  282.568274   
39  4.477211  2.729698  2.622131   6.631999   2.125036  2.209934  285.500354   
26  4.025323  2.361757  2.698339   7.529916   1.669574  2.392503  283.294322   
15  3.541635  1.243366  2.944033   6.492710   1.167544  3.373085  286.058698   

    Prec_o_avg  Hum_o_avg  
58    3.384098  74.405022  
16    3.063709  76.638941  
17    3.286370  77.866951  
96    3.380152  76.299782  
48    3.200523  75.556740  
..         ...        ...  
9     3.451143  76.023689  
32    3.758924  75.335186  
39    3.204750  76.043294  
26    3.367787  74.720056  
15    3.064636  76.158939  

[103 rows x 16 columns]

In [72]:
#Splitting dataset into testing and training set
train, test= train_test_split(final, test_size=0.30)

In [73]:
# Separating variabels and objective function values
X_train=train.iloc[:,2:]
Y_train=train.iloc[:,1:2]
X_test=test.iloc[:,2:]
Y_test=test.iloc[:,1:2]

In [75]:
model=LinearRegression()
model.fit(X_train.values, Y_train.values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [77]:
# Calculating coefficient of determination on training set
model.score(X_train.values,Y_train.values )

0.3586561841887334

In [81]:
# Calculating coefficient of determination on testing set
model.score(X_test.values,Y_test.values )

-8.328799816706637

In [82]:
#Predicting value on test dataset
Y_pred=model.predict(X_test.values)

In [85]:
#Calculating RMSE
np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))

0.4931984750884528